In [10]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [11]:
#parameters
maxlen = 30
labels = 2

In [12]:
czech = pd.read_excel('czech.xlsx', header = None)
czech.columns = ['female', 'male']

In [13]:
czech.head()

,female,male
0,Abigail,Abadon
1,Ada,Abdon
2,Adalberta,Ábel
3,Adéla,Abelard
4,Adelaida,Abraham


In [14]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [15]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [16]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
len_vocab = len(vocab)

In [17]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'END', 'q', '0', '2', 't', 'p', 'o', 'm', 'b', '9', 'a', 'j', 'y', '1', 'w', ' ', 'i', 'd', 's', 'l', 'c', 'k', '4', 'x', 'e', '5', '.', '3', 'f', 'z', 'g', '6', 'r', '7', 'v', 'n', 'h', '8', 'u'}
vocab length is  39
length of data_set is  15226


In [18]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [19]:
print(char_index)

{'n': 35, '6': 31, 'END': 0, 'c': 20, 'h': 36, '9': 9, 'q': 1, 'e': 24, '0': 2, 'z': 29, 'b': 8, 'x': 23, '3': 27, 'f': 28, '2': 3, 'g': 30, '4': 22, '1': 13, 't': 4, 'p': 5, 'o': 6, 'm': 7, 'u': 38, 'a': 10, '7': 33, 'r': 32, 'y': 12, 'v': 34, 'w': 14, ' ': 15, 's': 18, 'd': 17, 'j': 11, '8': 37, 'i': 16, 'l': 19, 'k': 21, '5': 25, '.': 26}


In [21]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.8
train = data_set1[msk]
test = data_set1[~msk]     

In [22]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
trunc_train_name = [str(i)[0:30] for i in train.name]
for i in trunc_train_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    train_X.append(tmp)

In [50]:
np.asarray(train_X).shape

(12214, 30)

In [52]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [53]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [54]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [55]:
train_X=np.asarray(train_X)

In [56]:
train_Y=np.asarray(train_Y)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [57]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...


In [58]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    

In [59]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(3012, 30, 39)
(3012, 2)


In [60]:
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)

In [61]:
batch_size=1000
model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=10,
          validation_data=(test_X, test_Y)
         )

Train on 12214 samples, validate on 3012 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor of shape [512,2048] and type float
	 [[node lstm_1/recurrent_kernel/initial_value (defined at /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:396)  = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [512,2048] values: [0.0496464074 -0.0121534793 -0.018046597...]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'lstm_1/recurrent_kernel/initial_value', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in star

In [49]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

AttributeError: 'list' object has no attribute 'ndim'

In [288]:
name=["sandhya","jaspreet","rajesh"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))

In [289]:
pred

array([[ 0.62356585,  0.37643418],
       [ 0.72094178,  0.27905828],
       [ 0.90337974,  0.09662029]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [290]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=50,validation_data=(test_X, test_Y))

Train on 12198 samples, validate on 3028 samples
Epoch 1/50
12198/12198 [==============================] - 145s - loss: 0.4107 - acc: 0.8137 - val_loss: 0.4408 - val_acc: 0.7966
Epoch 2/50
12198/12198 [==============================] - 144s - loss: 0.3912 - acc: 0.8254 - val_loss: 0.4479 - val_acc: 0.7936
Epoch 3/50
12198/12198 [==============================] - 145s - loss: 0.3927 - acc: 0.8228 - val_loss: 0.4511 - val_acc: 0.7982
Epoch 4/50
12198/12198 [==============================] - 145s - loss: 0.3730 - acc: 0.8344 - val_loss: 0.4253 - val_acc: 0.8071
Epoch 5/50
12198/12198 [==============================] - 145s - loss: 0.3640 - acc: 0.8396 - val_loss: 0.4240 - val_acc: 0.8164
Epoch 6/50
12198/12198 [==============================] - 145s - loss: 0.3490 - acc: 0.8505 - val_loss: 0.4183 - val_acc: 0.8180
Epoch 7/50
12198/12198 [==============================] - 145s - loss: 0.3411 - acc: 0.8542 - val_loss: 0.4089 - val_acc: 0.8243
Epoch 8/50
12198/12198 [========================

In [460]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3028/3028 [==============================] - 16s    
Test score: 0.448404541104
Test accuracy: 0.864266842879


<h3 align="center"> lets look at the loss and accuracy chart as a function of epochs </h3><img src="loss_charts.bmp" alt="loss charts" width="500" height="350"/><img src="acc_charts.bmp" alt="loss charts"  width="500" height="350"/>

In [342]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.0859881 ,  0.91401184],
       [ 0.96310365,  0.03689628],
       [ 0.7148453 ,  0.28515476],
       [ 0.02246205,  0.97753793],
       [ 0.13607673,  0.86392319],
       [ 0.99559009,  0.00440993],
       [ 0.05380283,  0.94619709],
       [ 0.55060732,  0.44939268],
       [ 0.10676169,  0.89323831]], dtype=float32)

In [345]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.15557961,  0.84442037],
       [ 0.25342518,  0.74657482],
       [ 0.8618474 ,  0.13815261]], dtype=float32)

In [502]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.33718896,  0.66281104],
       [ 0.99896383,  0.00103616],
       [ 0.99664474,  0.00335527]], dtype=float32)

In [450]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [464]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [479]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [483]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")